In [30]:
%%capture
%run 3-tfidf-gensim.ipynb

In [31]:
import gensim
from gensim import models

# %pip install pyldavis==3.3.1
import pyLDAvis
import pyLDAvis.gensim

In [32]:
# Fit an LDA model
lda_model = models.LdaModel(corpus=corpus, id2word=id2word, num_topics=20, passes=10, random_state=1)

In [33]:
# Visualize the LDA model using pyLDAvis
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary=lda_model.id2word, mds='tsne')
pyLDAvis.display(vis)

In [44]:
# Import necessary libraries
import pandas as pd

def format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=df_clean):
    # Init output
    sent_topics_df = pd.DataFrame(columns=['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords'])

    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df.loc[len(sent_topics_df)] = [int(topic_num), round(prop_topic, 4), topic_keywords]
            else:
                break
    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return sent_topics_df

df_topic_sents_keywords = format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=df_clean)


In [ ]:
df_topic_sents_keywords

In [50]:
df_topic_sents_keywords['Dominant_Topic'].unique()

array([ 9, 18, 13,  2, 17,  7, 16, 11,  3,  5, 12, 19,  4, 10,  6, 15,  8,
       14,  1,  0], dtype=int64)